In [1]:
%%html
<style>
  table { float: left; background: #d9d9d9; }
</style>

# find_cluster_by_name_datacenterメソッド

```python
def find_cluster_by_name_datacenter(datacenter, cluster_name):
```

datacenterのManaged Objectからcluster_nameで指定したManaged Objectを検索するメソッドです。  

## 引数

| args         | type                     | param                             |
| :-           | :----------------------- | :-------------------------------- |
| datacenter   | class                    | データセンターのManaged Object    |
| cluster_name | str                      | 検索するクラスター名              |

## 戻り値

| rtype             | return                                                                                      |
| :---------------- | :------------------------------------------------------------------------------------------ |
| class             | firstがTrueの時、nameにマッチしたManaged Objectを返す。もし、マッチしない場合はNoneを返す   |
| list              | firstがFalseの時listで結果を返す                                                            |

# find_cluster_by_name_datacenterメソッド動作確認

find_cluster_by_name_datacenterメソッドの動作確認をしてみます。  
以下の変数は環境に合わせて変更してください。

* dc_name
* cluster_name
* hostname
* username
* password
* validate_certs

In [ ]:
from ansible.module_utils.vmware import connect_to_api, find_datacenter_by_name, find_cluster_by_name_datacenter
from pyVmomi import vim

dc_name = 'Datacenter Name'
cluster_name = 'Cluster Name'

class module():
    def __init__(self):
        self.params = {}

m = module()
m.params['hostname'] = 'vCenter IP or HostName'
m.params['username'] = 'UserName'
m.params['password'] = 'Password'
m.params['validate_certs'] = False

sc = connect_to_api(m)
dc_obj = find_datacenter_by_name(sc, dc_name)
if(dc_obj):
    cluster_obj = find_cluster_by_name_datacenter(dc_obj, cluster_name)
    if(cluster_obj):
        print(cluster_obj.name)
    else:
        print("%s not found" % cluster_name)
else:
    print("%s not found" % dc_name)

# find_cluster_by_name_datacenterメソッドを使ったモジュール例

find_cluster_by_name_datacenterを使って実際にAnsibleモジュールを作ってみましょう。  
以下のモジュールはクラスタ名を指定してフィルタリングされた結果を戻すだけの単純なものです。  
モジュールは `/tmp/ansible_salf_made_module` に `get_cluster_name.py` で保存されます。  
`/tmp/ansible_salf_made_module` ディレクトリが無い場合は作成してください。  
既にディレクトリがある場合は、以下を実行してモジュールファイルを作成します。

In [ ]:
%%writefile /tmp/ansible_salf_made_module/get_cluster_name.py
#!/usr/bin/python

try:
    from pyVmomi import vim, vmodl
    HAS_PYVMOMI = True
except ImportError:
    HAS_PYVMOMI = False

from ansible.module_utils.vmware import find_datacenter_by_name, find_cluster_by_name_datacenter, connect_to_api, vmware_argument_spec
from ansible.module_utils.basic import AnsibleModule

def main():
    argument_spec = vmware_argument_spec()
    argument_spec.update(dc_name=dict(type='str', required=True),
                        cluster_name=dict(type='str', required=True))

    module = AnsibleModule(argument_spec=argument_spec, supports_check_mode=True)

    if not HAS_PYVMOMI:
        module.fail_json(msg='pyvmomi python library not found')

    dc_name = module.params['dc_name']
    cluster_name = module.params['cluster_name']

    result = dict(changed=False)
    sc = connect_to_api(module)
    dc_obj = find_datacenter_by_name(sc, dc_name)
    if(dc_obj):
        cluster_obj = find_cluster_by_name_datacenter(dc_obj, cluster_name)
        if(cluster_obj):
            result['cluster'] = cluster_obj.name
            module.exit_json(**result)
        else:
            module.fail_json(msg="%s not found" % cluster_name)
    else:
        module.fail_json(msg="%s not found" % dc_name)

if __name__ == "__main__":
    main()

# モジュールを動かす

作ったモジュールを実際に動かしてみます。  
以下のオプションは環境に合わせて変更してください。

* hostname
* username
* password
* validate_certs
* dc_name
* cluster_name

変更後、Kernelを `Ansible` に変更して以下を実行します。

In [ ]:
#ansible.cfg
[defaults]
library = /tmp/ansible_salf_made_module

In [ ]:
#inventory
[all]
localhost

In [ ]:
#play
name: salf-made module test
hosts: localhost
connection: local
gather_facts: no

In [ ]:
#task
get_cluster_name:
  hostname: vCenter IP or HostName
  username: UserName
  password: Password
  validate_certs: no
  dc_name: Datacenter Name
  cluster_name: Cluster Name

`name` に指定したVM名が取得できていることを確認します。